# FMZ Übung 7: **Kreuzvalidierung & Bootstrap**

---
### Ziel der Übung
- Überanpassung („Overfitting“) sichtbar machen
- Training/Test-Aufteilung anwenden
- Bootstrap-Resampling für Schätzung der Fehlervariabilität nutzen
- Mehrere Fehlermetriken berechnen
- Modelle: Lineare Regression, Polynomregression, Random Forest, SVM, Neural Net


---


In [ ]:
# Benötigte Pakete

install.packages(c("ggplot2","dplyr","randomForest","e1071","neuralnet"))

library(ggplot2)
library(dplyr)
library(randomForest)
library(e1071)
library(neuralnet)


---
## Datensatz erzeugen (synthetisch, nichtlinear)

In [ ]:

set.seed(1)
n <- 500
x <- runif(n, -5, 5)
y <- sin(x) + rnorm(n, sd=.5)
data <- data.frame(x,y)
plot(data)


---
## 1. Aufgabe: **Train/Test-Aufteilung & Overfitting**

- Erstellen Sie einen 80/20-Datenaufteilung und passen Sie drei Modelle an:
  - lineare Regression
  - Polynomregression
  - Random Forest

- Berechnen Sie die R²-Fehlermetrik

- Vergleichen Sie Train vs Test, um Overfitting zu zeigen.

In [ ]:

# Aufteilung der Daten
set.seed(1)
id <- sample(1:nrow(data), 0.8*nrow(data))
train <- data[id,]
test  <- data[-id,]  #-id: Tipp, um die verbleibenden Beobachtungen zu erhalten

#R²-Fehlermetrik
R2   <- function(y, yhat) 1 - sum((y-yhat)^2)/sum((y-mean(y))^2)

# 1) Lineare Regression
mod_lin <- lm(y ~ x, data=train)

# 2) Polynomregression
mod_poly <- lm(y ~ poly(x,5), data=train)

# 3) Random Forest
mod_rf <- randomForest(y ~ x, data=train, ntree=200)

# Darstellung der Ergebnisse
evaluate <- function(model, train, test)
  tibble(
    R2_train   = R2(train$y, predict(model, train)),
    R2_test    = R2(test$y,  predict(model, test)))

results <- bind_rows(
  evaluate(mod_lin, train, test)  |> mutate(Model="Linear"),
  evaluate(mod_poly, train, test) |> mutate(Model="Poly"),
  evaluate(mod_rf, train, test)   |> mutate(Model="RF"))

print(results)


In [ ]:

# Graphische Darstellung der Ergebnisse
model_list <- list("Linear" = mod_lin, "Polynom" = mod_poly, "Random Forest" = mod_rf)

for(name in names(model_list)){
  mod <- model_list[[name]]

  # Vorhersagen
  yt_tr <- predict(mod, train)
  yt_ts <- predict(mod, test)

  # Plot
  plot(train$x, train$y, pch=19, col="grey60", main=name, xlab="x", ylab="y")
  points(train$x, yt_tr, col="blue", pch=16)
  points(test$x, test$y, col="grey60", pch=1)
  points(test$x, yt_ts, col="red", pch=16)
  legend("bottomleft", legend=c(paste("Train R² =", round(R2(train$y, yt_tr),3)), paste("Test R² =", round(R2(test$y, yt_ts),3))), bty="n")
}

---
## 2. Aufgabe: **Bootstrap-Cross-Validation**

Jetzt wird eine Bootstrap-Schleife (B=100) implementiert.
In jeder Iteration:
- Zufällige Ziehung von 80% der Daten als Training
- Modell schätzen
- Training/Test-Fehler speichern

So erhalten wir Verteilungen für die Fehlermetriken statt Punktschätzungen.

In [ ]:

# Speichern der Fehlermetriken jeder Iteration
B <- 100
store <- data.frame(
  R2_test_lin = numeric(B),
  R2_test_poly = numeric(B),
  R2_test_rf = numeric(B))

set.seed(1)
for(b in 1:B){

  # Train/Test-Aufteilung der Daten
  id <- sample(1:nrow(data), 0.8*nrow(data))
  train <- data[id,]
  test  <- data[-id,]

  # Modelle
  m_lin  <- lm(y ~ x, data=train)
  m_poly <- lm(y ~ poly(x,8), data=train)
  m_rf   <- randomForest(y ~ x, data=train, ntree=200)

  # Fehler speichern
  store$R2_test_lin[b]  <- R2(test$y, predict(m_lin, test))
  store$R2_test_poly[b] <- R2(test$y, predict(m_poly, test))
  store$R2_test_rf[b]   <- R2(test$y, predict(m_rf, test))
}

# Graphische Darstellung der R²-Verteilungen (Boxplot + Histogramm)
df_long <- data.frame(
  Model = rep(names(store), each = nrow(store)),
  R2    = as.vector(as.matrix(store)))

ggplot(df_long, aes(x = Model, y = R2, fill = Model)) +
  geom_boxplot() +
  theme_bw()

ggplot(df_long, aes(R2, fill = Model)) +
  geom_histogram(alpha=.5, bins=30, position="identity") +
  theme_bw()


---
## 3. Aufgabe: **Vergleich unterschiedlicher Fehlermetriken**

- RMSE (root mean square error)
- MAE (mean absolute error)
- R² (Bestimmtheitsmaß)

In [ ]:

# Fehlermetriken
RMSE <- function(y, yhat) sqrt(mean((y - yhat)^2))
MAE  <- function(y, yhat) mean(abs(y - yhat))
R2   <- function(y, yhat) 1 - sum((y - yhat)^2) / sum((y - mean(y))^2)

# Beispiel für Random Forest
set.seed(2)
id <- sample(1:nrow(data), 0.8*nrow(data))
  train <- data[id,]
  test  <- data[-id,]

  m <- randomForest(y ~ x, data=train, ntree=500)
  #m <- lm(y ~ poly(x,8), data=train)
  pred_train <- predict(m, train)
  pred_test <- predict(m, test)

  print(paste("RMSE-Train:", round(RMSE(train$y, pred_train), 5)))
  print(paste("MAE-Train:", round(MAE(train$y, pred_train), 5)))
  print(paste("R²-Train:", round(R2(train$y, pred_train), 5)))

  print(paste("RMSE-Test:", round(RMSE(test$y, pred_test), 5)))
  print(paste("MAE-Test:", round(MAE(test$y, pred_test), 5)))
  print(paste("R²-Test:", round(R2(test$y, pred_test), 5)))


---
## 4. Aufgabe: **Hyperparameter-Optimierung**

- Wählen Sie:
   - Polynomialgrad: 3, 5, 8
   - Random Forest: ntree = 50, 200, 500

- Verwende eine verschachtelte Bootstrap-Schleife, um die optimalen Hyperparameter zu schätzen.

In [ ]:

# Wahl des Polynomgrads und der Baumanzahl
set.seed(1)
poly_deg  <- c(3,5,8)
rf_ntree  <- c(50,200,500)

# Speichern der Fehlermetriken
best_poly <- numeric(B)
best_rf   <- numeric(B)

# Bootstrap-Schleife
B <- 100
for(b in 1:B){
  id  <- sample(1:nrow(data), 0.8*nrow(data))
  tr   <- data[id,]
  ts   <- data[-id,]

  # Polynomial
  rm <- sapply(poly_deg, function(d){
    m <- lm(y ~ poly(x,d), data=tr)
    RMSE(ts$y, predict(m, ts)) })
  best_poly[b] <- poly_deg[ which.min(rm) ]

  # Random Forest
  rm <- sapply(rf_ntree, function(nt){
    m <- randomForest(y ~ x, data=tr, ntree=nt)
    RMSE(ts$y, predict(m, ts)) })
  best_rf[b] <- rf_ntree[ which.min(rm) ]
}

table(best_poly)
table(best_rf)


---
## Zusatzaufgabe: **Anwendung auf realen Daten I**

- Verwendung des Datensatzes `mtcars`:
   - y = `mpg`
   - x = `hp`, `wt`, `disp`

- Setzen Sie Bootstrap-Cross-Validation ein, um die Genauigkeit mehrerer Modelle zu vergleichen.

In [ ]:

# Aufbau des Datensatzes
set.seed(1)
summary(mtcars)
df <- mtcars[,c("mpg","hp","wt","disp")]
B <- 100

# Speichern der Fehlermetriken
store_RMSE <- data.frame(
  lin=numeric(B),
  rf=numeric(B),
  svm=numeric(B))
store_R2 <- data.frame(
  lin=numeric(B),
  rf=numeric(B),
  svm=numeric(B))

# Bootstrap-Schleife
for(b in 1:B){
  idx <- sample(1:nrow(df), 0.8*nrow(df), replace=TRUE)
  tr  <- df[idx,]
  ts  <- df[-unique(idx),]

  # Linear
  m1 <- lm(mpg ~ ., data=tr)
  store_RMSE$lin[b] <- RMSE(ts$mpg, predict(m1, ts))
  store_R2$lin[b] <- R2(ts$mpg, predict(m1, ts))

  # Random Forest
  m2 <- randomForest(mpg ~ ., data=tr, ntree=300)
  store_RMSE$rf[b] <- RMSE(ts$mpg, predict(m2, ts))
  store_R2$rf[b] <- R2(ts$mpg, predict(m2, ts))

  # SVM
  m3 <- svm(mpg ~ ., data=tr, cost=1)
  store_RMSE$svm[b] <- RMSE(ts$mpg, predict(m3, ts))
  store_R2$svm[b] <- R2(ts$mpg, predict(m3, ts))
}

boxplot(store_RMSE, main="Bootstrap RMSE – Vergleich (mtcars)")
boxplot(store_R2, main="Bootstrap R² – Vergleich (mtcars)")


---
## Zusatzaufgabe: **Anwendung auf realen Daten II**

Fußgänger-Datensatz (Vorlesungsfolien)

In [ ]:

# Lesen der Daten
ped <- read.table("https://raw.githubusercontent.com/antoinetordeux/Datasets/refs/heads/main/ped_data.txt", header=TRUE)
head(ped)
attach(ped)

# Graphische Darstellung der Ergebnisse
par(mfrow=c(2,2))
plot_res=function(algo,titel,cc){
	for(tt in c(titel, "Testing")){
		plot(X[cc,1], Y[cc], xlab="X: Spacing (m)", ylab="Y: Speed (m/s)", main=tt)
		lines(sort(Spacing[cc]), predict(algo,as.data.frame(X[cc,]))[order(Spacing[cc])], col="blue", lwd=1)
		legend("bottomright", paste('R² =', round(R2(Y[cc],predict(algo,as.data.frame(X[cc,]))), 3)), bty='n')
		cc=!cc}}


# Datenaufteilung
X=cbind(Spacing,Spacing_pred,Speed_pred,Acceleration)
Y=Speed
n=length(Y)
cc=NULL;cc[1:n]=T;cc[sample(1:n,.2*n)]=F

# Multiple lineare Regression
algo_MLR=lm(Y[cc]~.,data=as.data.frame(X[cc,]))
plot_res(algo_MLR,expression(paste(italic(Y)==italic(a[1]*X[1]+...+a[n]*X[n]+b))),cc)

# Neural network
algo_NN=neuralnet(Y[cc]~.,data=X[cc,],h=c(1,2))
plot_res(algo_NN,expression("Neural network c(1,2)"),cc)

# Random Forest
algo_RF=randomForest(Y[cc]~.,data=X[cc,])
plot_res(algo_RF,expression("Random forest"),cc)

# Support Vector Machine
algo_SVM=svm(Y[cc]~.,data=X[cc,])
plot_res(algo_SVM,expression("Support vector machine"),cc)

detach()
